In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
os.environ["KERAS_BACKEND"] = "torch"
import keras
from datamodules import data_processors
import sys
sys.path.append("..")
print(sys.version)
from rusmodules.inverse import inverse_standard, get_constants

3.13.1 (main, Dec  9 2024, 00:00:00) [GCC 14.2.1 20240912 (Red Hat 14.2.1-3)]


In [2]:
path_modelo = "models/cubico_L1.keras"
modelo = keras.models.load_model(path_modelo)

In [3]:
path_archivo = "../input_data/cubic_1.csv"
full_data = pd.read_csv(path_archivo)
full_data.head()

,phi_a,x_K,eta,beta,phi_K,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_13,eig_14,eig_15,eig_16,eig_17,eig_18,eig_19,Parallelepiped,Cylinder,Ellipsoid
0,0.843666,0.647473,1.577259,1.574131,0.866533,0.552390,2.341186,2.982731,4.346947,4.543615,...,10.255629,10.934964,12.169362,12.639375,13.023291,17.260278,18.912029,1,0,0
1,0.618911,0.218057,1.216162,1.820191,1.350973,0.451902,1.069440,2.198825,3.912317,3.934114,...,11.230938,11.241764,11.468446,14.561622,14.919805,15.338283,15.666733,1,0,0
2,0.076877,0.245986,1.434582,2.036289,1.322259,0.104398,3.832743,4.758215,5.732612,5.807000,...,15.088024,15.429422,16.241738,16.394792,20.165109,25.500514,26.305482,1,0,0
3,1.459619,0.517388,1.481719,1.376496,1.027000,0.202136,2.142934,2.641222,2.779140,3.088002,...,10.969815,12.756643,13.922951,14.045571,15.047647,18.770442,19.392401,1,0,0
4,0.032508,0.949501,1.839938,0.803203,0.319154,0.002714,10.061990,13.963490,14.997477,17.405756,...,56.063736,70.470168,72.668095,76.780522,93.416045,97.561829,106.012724,1,0,0


In [4]:
N_eig = 19
features = ["eta", "beta"] + list(map(lambda x: "eig_" + str(x), range(N_eig + 1)))
target = ["C00", "C01", "C33"]

In [5]:
full_data["x_a"] = np.sin(full_data["phi_K"])*np.cos(full_data["phi_a"])
full_data["x_mu"] = np.sin(full_data["phi_K"])*np.sin(full_data["phi_a"])
full_data["C00"] = full_data["x_K"] + 2*full_data["x_a"]
full_data["C01"] = full_data["x_K"] - full_data["x_a"]
full_data["C33"] = full_data["x_mu"]
X_data = full_data[features]
y_data = full_data[target]

In [6]:
X_data.head()

,eta,beta,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,...,eig_10,eig_11,eig_12,eig_13,eig_14,eig_15,eig_16,eig_17,eig_18,eig_19
0,1.577259,1.574131,0.552390,2.341186,2.982731,4.346947,4.543615,4.679809,4.958647,5.785759,...,8.788997,8.857167,10.043210,10.255629,10.934964,12.169362,12.639375,13.023291,17.260278,18.912029
1,1.216162,1.820191,0.451902,1.069440,2.198825,3.912317,3.934114,4.720611,4.783530,5.282552,...,8.470418,8.802802,11.027353,11.230938,11.241764,11.468446,14.561622,14.919805,15.338283,15.666733
2,1.434582,2.036289,0.104398,3.832743,4.758215,5.732612,5.807000,7.884011,8.074111,9.225926,...,10.754743,12.831688,14.911388,15.088024,15.429422,16.241738,16.394792,20.165109,25.500514,26.305482
3,1.481719,1.376496,0.202136,2.142934,2.641222,2.779140,3.088002,4.152219,4.371824,6.257074,...,8.846824,8.970201,10.053547,10.969815,12.756643,13.922951,14.045571,15.047647,18.770442,19.392401
4,1.839938,0.803203,0.002714,10.061990,13.963490,14.997477,17.405756,24.931523,26.047636,29.607472,...,41.387717,41.568790,54.541584,56.063736,70.470168,72.668095,76.780522,93.416045,97.561829,106.012724


In [7]:
y_data.head()

,C00,C01,C33
0,1.660636,0.140891,0.569345
1,1.807878,-0.576853,0.566188
2,2.178807,-0.720424,0.074442
3,0.707277,0.422444,0.850468
4,1.576696,0.635904,0.010198


In [8]:
X_sample = X_data.iloc[3]
y_sample = y_data.iloc[3]

In [9]:
eigs = np.array(X_sample[list(map(lambda x: "eig_" + str(x),range(N_eig + 1)))])
eigs

array([ 0.20213604,  2.14293354,  2.64122179,  2.77914025,  3.08800192,
        4.15221862,  4.37182384,  6.25707376,  6.33860889,  8.38991043,
        8.84682353,  8.97020071, 10.05354668, 10.96981511, 12.75664293,
       13.92295087, 14.0455707 , 15.04764682, 18.77044158, 19.392401  ])

In [10]:
eta = X_sample["eta"]
beta = X_sample["beta"]
eta, beta

(np.float64(1.4817193760885254), np.float64(1.37649557470242))

In [11]:
Constants = get_constants(eigs, eta, beta, modelo, include_x0=False)

{'eta': np.float64(1.4817193760885254), 'beta': np.float64(1.37649557470242), 'x_1': np.float64(0.058937185551530304), 'x_2': np.float64(0.02569502652661284), 'x_3': np.float64(0.00711198464029025), 'x_4': np.float64(0.015926943069220347), 'x_5': np.float64(0.05487802656980103), 'x_6': np.float64(0.011324292538404746), 'x_7': np.float64(0.09721591068405634), 'x_8': np.float64(0.004204488595289967), 'x_9': np.float64(0.10577862666809466), 'x_10': np.float64(0.02356145034065119), 'x_11': np.float64(0.006362140398479911), 'x_12': np.float64(0.05586445821427598), 'x_13': np.float64(0.047248838621824274), 'x_14': np.float64(0.09214061862973744), 'x_15': np.float64(0.06014252376440024), 'x_16': np.float64(0.006323086571392967), 'x_17': np.float64(0.05167364890784636), 'x_18': np.float64(0.19197183298127843), 'x_19': np.float64(0.032072326274662624)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


/home/cubos/.local/lib/python3.13/site-packages/keras/src/backend/torch/numpy.py:114: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:328.)
  return cast(torch.matmul(x1, x2), result_dtype)


In [12]:
Constants

{'C00': np.float64(0.6637212920769728),
 'C01': np.float64(0.37713595848447945),
 'C33': np.float64(0.8430677748026503)}